In [1]:
'''
Citation: https://github.com/hamzarawal/HMM-Viterbi/blob/master/viterbi.py
'''
import numpy as np

#generating initial probabilities

#transition probabilities
transition = np.array([[0.8,0.1],
                       [0.1,0.8]])
#Emission probabilities
emission = np.array([[0.1,0.2,0.7],
                     [0.7,0.2,0.1]])

#defining states and sequence symbols
states = ['H','C']
states_dic = {'H':0, 'C':1}
sequence_syms = {'1':0,'2':1,'3':2}

#test sequence
test_sequence = '111133333'
test_sequence = [x for x in test_sequence]
print(test_sequence)


#node values stored during viterbi forward algorithm
node_values = np.zeros((len(states),len(test_sequence)))

#probabilities of going to end state
end_probs = [0.1, 0.1]

#probabilities of going from start state
start_probs = [0.5, 0.5]

#storing max symbol for each stage
max_syms = np.chararray((len(states),len(test_sequence)))

for i, sequence_val in enumerate(test_sequence):
    
    for j in range(len(states)):
        #if first sequence value then do this
        
        if(i == 0):
            node_values[j,i] = start_probs[j]*emission[j,sequence_syms[sequence_val]]
            #print('node_values[j,i] is',node_values[j,i])
            
        #else perform this
        else:
            values = [node_values[k, i - 1] * emission[j, sequence_syms[sequence_val]] * transition[k, j] for k in
                      range(len(states))]

            max_idx = np.argmax(values)
            max_val = max(values)
            max_syms[j,i] = states[max_idx]
            node_values[j,i] = max_val
            
print('max_syms are...',max_syms)

#end state value
end_state = np.multiply(node_values[:,-1],end_probs)
end_state_val = max(end_state)
end_state_max_idx = np.argmax(end_state)
end_state_sym = states[end_state_max_idx]

print("Node values are....", node_values)

#Obtaining the maximum likely states
max_likely_states = [end_state_sym]
prev_max = end_state_sym

for count in range(1,len(test_sequence)):
    current_state = max_syms[:,-count][states_dic[prev_max]].decode('utf-8')
    max_likely_states.append(current_state)
    prev_max = current_state
    
max_likely_states = max_likely_states[::-1]

c = 1

print(max_likely_states)


['1', '1', '1', '1', '3', '3', '3', '3', '3']
max_syms are... [[b'\x03' b'H' b'C' b'C' b'C' b'H' b'H' b'H' b'H']
 ['' b'C' b'C' b'C' b'C' b'C' b'C' b'H' b'H']]
Node values are.... [[5.00000000e-02 4.00000000e-03 1.96000000e-03 1.09760000e-03
  4.30259200e-03 2.40945152e-03 1.34929285e-03 7.55603997e-04
  4.23138238e-04]
 [3.50000000e-01 1.96000000e-01 1.09760000e-01 6.14656000e-02
  4.91724800e-03 3.93379840e-04 3.14703872e-05 1.34929285e-05
  7.55603997e-06]]
['C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H']


In [2]:
#transition probabilities
tpm = np.array([[0.8,0.1],[0.1,0.8]])

#Emission probabilities
epm = np.array([[0.1,0.2,0.7],[0.7,0.2,0.1]])

#start prob
pi = [0.5, 0.5]

#test sequence
observations = [1,1,1,1,3,3,3,3,3]

In [10]:
import numpy
def our_viterbi(tpm, epm, pi, observations):
    
    num_states = 2
    states_names = ['H','C']
    
    # create path probability matrix num observations by num states
    # initialize to 0
    path_probability_matrix = numpy.zeros((len(observations), num_states ))

    #create a path backpointer matrix backpointer[N, L + 2] to save indexes of states
    # initialize to 0
    path_backpointer_maxtrix = numpy.zeros((len(observations), num_states ))
    
    max_states = np.zeros(len(observations))
    #final_path = np.zeros(len(observations))

    final_path=[]
    final_path = ['starting' for i in range(len(observations))]
    print('finalpath is', final_path)

    #grab first observation to determine which EPM index to use
    if observations[0] == 1:
        obs_index = 0
    elif observations[0] == 2:
        obs_index = 1
    else:
        obs_index = 2
        
    # update first row of path_probability_matrix matrix
    # first row = probability of starting in each state * prob of seeing observation in that state
    for s in range(num_states):
        path_probability_matrix[0][s] = pi[s] * epm[s][obs_index]
    
    for t in range(1, len(observations)):
        for s in range(num_states):
            
            backpointer_probabilities = []
            probabilities = []
            
            for s_prime in range(num_states):
                
                # prob of getting to each previous state through all possible paths
                a = path_probability_matrix[t-1][s_prime]
                
                # the transition probability from previous S to current S
                b = tpm[s_prime][s]
                
                backpointer_prob = a * b
                backpointer_probabilities.append(backpointer_prob)
                
                # the emission probability of emitting observation at current S
                if observations[t] == 1:
                    c = epm[s][0]
                elif observations[t] == 2:
                    c = epm[s][1]
                else:
                    c = epm[s][2]
                    
                path_probability = a * b * c
                probabilities.append(path_probability)
        
            # update path_probability_matrix[t,s] to MAX probability of most prob state for previous observation
            path_probability_matrix[t][s] = max(probabilities)

            # update back_pointer[t, s] to be argMax calculated of previous obs for given observation
            path_backpointer_maxtrix[t][s] = numpy.argmax(backpointer_probabilities)
            
    print("final path_probability_matrix is", path_probability_matrix)
    
    #grab end state probability and index 
    end_state_probability = max(path_probability_matrix[-1:])
    end_state_index = numpy.argmax(path_probability_matrix[-1:])
    end_state_name = states_names[end_state_index]
    
    #backtrack through the backpointer matrix, starting with the most probable last state calculated
    print('end_state_index is..', end_state_index, 'and finalpath is',final_path)
    
    #go backwards through backpointer
    print('path_probability_matrix \n', path_probability_matrix)

    print("\n before loop path_backpointer_maxtrix", path_backpointer_maxtrix)
    final_path[len(observations)-1] = end_state_index

    for i in range(len(observations)-1, -1, -1):
        final_path_state = path_backpointer_maxtrix[i][end_state_index]
        final_path[i] = final_path_state
        end_state_index = int(final_path_state)

    print(final_path)
        
        
    # should be 1,1,1,1,0,0,0,0,0
    

In [11]:
our_viterbi(tpm, epm, pi, observations)

finalpath is ['starting', 'starting', 'starting', 'starting', 'starting', 'starting', 'starting', 'starting', 'starting']
final path_probability_matrix is [[5.00000000e-02 3.50000000e-01]
 [4.00000000e-03 1.96000000e-01]
 [1.96000000e-03 1.09760000e-01]
 [1.09760000e-03 6.14656000e-02]
 [4.30259200e-03 4.91724800e-03]
 [2.40945152e-03 3.93379840e-04]
 [1.34929285e-03 3.14703872e-05]
 [7.55603997e-04 1.34929285e-05]
 [4.23138238e-04 7.55603997e-06]]
end_state_index is.. 0 and finalpath is ['starting', 'starting', 'starting', 'starting', 'starting', 'starting', 'starting', 'starting', 'starting']
path_probability_matrix 
 [[5.00000000e-02 3.50000000e-01]
 [4.00000000e-03 1.96000000e-01]
 [1.96000000e-03 1.09760000e-01]
 [1.09760000e-03 6.14656000e-02]
 [4.30259200e-03 4.91724800e-03]
 [2.40945152e-03 3.93379840e-04]
 [1.34929285e-03 3.14703872e-05]
 [7.55603997e-04 1.34929285e-05]
 [4.23138238e-04 7.55603997e-06]]

 before loop path_backpointer_maxtrix [[0. 0.]
 [0. 1.]
 [1. 1.]
 [1. 1.]

In [9]:
'''
in ours - we include  first backpointers as 0,0 ...in test they have it empty 
backpointer matrix is 
[[0. 0.]
 [0. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [0. 1.]
 [0. 1.]
 [0. 0.]
 [0. 0.]]

max_syms are... 
[['' b'H' b'C' b'C' b'C' b'H' b'H' b'H' b'H']
 ['' b'C' b'C' b'C' b'C' b'C' b'C' b'H' b'H']]
 
 '''

"\nin ours - we include  first backpointers as 0,0 ...in test they have it empty \nbackpointer matrix is \n[[0. 0.]\n [0. 1.]\n [1. 1.]\n [1. 1.]\n [1. 1.]\n [0. 1.]\n [0. 1.]\n [0. 0.]\n [0. 0.]]\n\nmax_syms are... \n[['' b'H' b'C' b'C' b'C' b'H' b'H' b'H' b'H']\n ['' b'C' b'C' b'C' b'C' b'C' b'C' b'H' b'H']]\n \n "